Новая структура парсера.

1. Получаем ссылки на города.
    1. Можно получить информацию с помощью гет запроса. (менее 1 мни/город)
    2. Требуется куки браузера что бы пройти авторизацию (получается при заходе на страницу через браузер)
2. Получаем информацию о магазинах
    1. get "location_info" выдает информацию об id ключах магазинов. (менее 1 мни/город)
    2. Так же требуется куки браузера
3. Получаем информацию об категориях и субкатегориях
    1. Пока единственный способ - это использовать Selenium. (1-2 час/город)
    2. Необходимо узнать как еще можно получить эту информацию из запросов.
4. Получаем информацию о товарах и скидке через Get запросы.
    1. get "item" выдает информацию об скидках и товарах (15-30 мин/город).
    2. Максимальное количество товара в запросе 600 единиц,
    3. Так же 600 единиц это максимальное количество товара, которое можно ВООБЩЕ получить.
    4. Что бы обойти это ограничение и требуется максимальная детализация в категориях/субкатегория
5. Получение Полной информации о товаре.
    1. в очереди

In [15]:
import pickle
from typing import NamedTuple
import pandas as pd
from urllib.parse import urlparse
from urllib.parse import parse_qs
import numpy as np
import requests
from requests.exceptions import ConnectTimeout
from tqdm import tqdm
import sys
from multiprocessing.dummy import Pool as ThreadPool
from time import sleep as time_sleep
tqdm.pandas()
requests.packages.urllib3.disable_warnings()

In [16]:
class market_link(NamedTuple):
    sity: str
    title: str
    discounts: int
    alcohol: bool
    href: str
    su1: str
    su2: str
    su3: str
    page_href: str

In [17]:
with open('data/market_catalog_links.pickle', 'rb') as file:
    market_catalog_links = pickle.load(file)

In [18]:
df = pd.DataFrame(market_catalog_links)

In [22]:
df.iloc[0]

sity                                                   lipetsk
title                                              Перекрёсток
discounts                                                 1682
alcohol                                                  False
href           https://edadeal.ru/lipeck/retailers/perekrestok
su1                                                   Продукты
su2                                          Молочные продукты
su3                                                        Сыр
page_href    https://edadeal.ru/lipeck/retailers/perekresto...
Name: 0, dtype: object

In [5]:
def get_params(ceil):
    
    parsed_url = urlparse(ceil)
    params = parse_qs(parsed_url.query)
    key = list(params.keys())
    try:
        return params.get(key[1], np.nan)[0]
    except IndexError:
        try:
            return params.get(key[0], np.nan)[0]
        except IndexError:
            return np.nan

df['su_id'] = df['page_href'].apply(get_params)
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3653 entries, 0 to 3652
Data columns (total 10 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   sity       3653 non-null   object
 1   title      3653 non-null   object
 2   discounts  3653 non-null   int64 
 3   alcohol    3653 non-null   bool  
 4   href       3653 non-null   object
 5   su1        3653 non-null   object
 6   su2        3653 non-null   object
 7   su3        3002 non-null   object
 8   page_href  3653 non-null   object
 9   su_id      3653 non-null   object
dtypes: bool(1), int64(1), object(8)
memory usage: 260.5+ KB


## Запрос

In [6]:
headers = {
# "Accept-Encoding": "gzip, deflate, br",
# "Connection": "keep-alive",
# "Content-Type": "application/json",
# "Host": "search.edadeal.io",
# "Origin": "https://edadeal.ru",
# "Referer": "https://edadeal.ru/",
# "Sec-Fetch-Dest": "empty",
# "Sec-Fetch-Mode": "cors",
# "Sec-Fetch-Site": "cross-site",
# "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/116.0.0.0 Safari/537.36",
# "accept": "application/json",
# "accept-language": "ru",
# "edadeal-duid": "4eef8e3c-2b51-4fb5-90c4-33153842365b",
# "x-app-id": "edadeal",
# "x-app-version": "1.52.0",
"x-locality-countrygeoid": "225",
"x-locality-geoid": "9",
"x-os-version": "1.0.0",
"x-platform": "desktop",
"x-position-latitude": "52.60882",
"x-position-longitude": "39.59922"
          }

In [55]:
params = {  'addContent': ['true'],
            'checkAdult': ['true'],
            'excludeSegmentSlug': ['alcohol', 'pt_alcool', 'en_alcohol', 'es_alcohol', 'tr_alkol'],
            'groupBy': ['sku_or_meta'],
            'numdoc': ['599'],
            'page': ['0'],
            'segmentUuid': ['3b31b30e-6311-11e6-849f-52540010b608']}

In [105]:
def get_price(ceil):
    if pd.isnull(ceil):
        return np.nan
        
    params = {  'addContent': ['true'],
            'checkAdult': ['true'],
            'excludeSegmentSlug': ['alcohol', 'pt_alcool', 'en_alcohol', 'es_alcohol', 'tr_alkol'],
            'groupBy': ['sku_or_meta'],
            'numdoc': ['599'],
            'page': ['0'],
            'segmentUuid': ['3b31b30e-6311-11e6-849f-52540010b608']}
    params['segmentUuid'] = [ceil]
    try:
        respons = requests.get('https://search.edadeal.io/api/v4/retailer/94d987b8-42c3-11e6-9419-52540010b608/items', params=params, headers=headers, verify=False)
    except Exception:
        exc_type, exc_obj, exc_tb = sys.exc_info()
        print(exc_type.__name__)
        return np.nan
    return respons.json()

In [106]:
df['data'] = (df['su_id']).progress_apply(get_price)

 84%|███████████████████████████████████████████████████████████████▉            | 3075/3653 [14:57<1:05:24,  6.79s/it]

ConnectTimeout


100%|██████████████████████████████████████████████████████████████████████████████| 3653/3653 [17:12<00:00,  3.54it/s]


In [9]:
def get_price_pool(pool_input: tuple[str, tqdm]):
    ceil, pbar = pool_input
    if pd.isnull(ceil):
        pbar.update(1)
        return (ceil, np.nan)
        
    params = {  'addContent': ['true'],
            'checkAdult': ['true'],
            'excludeSegmentSlug': ['alcohol', 'pt_alcool', 'en_alcohol', 'es_alcohol', 'tr_alkol'],
            'groupBy': ['sku_or_meta'],
            'numdoc': ['599'],
            'page': ['0'],
            'segmentUuid': ['3b31b30e-6311-11e6-849f-52540010b608']}
    params['segmentUuid'] = [ceil]
    for _ in range(10):
        try:
            respons = requests.get('https://search.edadeal.io/api/v4/retailer/94d987b8-42c3-11e6-9419-52540010b608/items', params=params, headers=headers, verify=False)
        except ConnectTimeout:
            time_sleep(1)
            continue
        except ChunkedEncodingError:
            time_sleep(1)
            continue
        except Exception:
            exc_type, exc_obj, exc_tb = sys.exc_info()
            print(exc_type.__name__)
            pbar.update(1)
            return (ceil, np.nan)
        pbar.update(1)
        return (ceil, respons.json())
    else:
        (ceil, 'timeout')

def run_get_price(list_input):
    with tqdm(total=len(list_input) ) as pbar:
        list_to_pool = list(zip(list_input, ([pbar]*len(list_input))))
        with ThreadPool(3) as pool:
            work_return = pool.map(get_price_pool, list_to_pool)
    return work_return

In [8]:
result = run_get_price(df['su_id'].tolist())

 99%|█████████████████████████████████████████████████████████████████████████████▎| 3623/3653 [06:00<00:02, 11.44it/s]

ChunkedEncodingError


100%|██████████████████████████████████████████████████████████████████████████████| 3653/3653 [06:07<00:00,  9.95it/s]


In [ ]:
tmp = pd.DataFrame(result, columns=[0,'data'])
test = df.merge(right=tmp,
                how='left',
                left_on='su_id',
                right_on=0,
                )#.drop(columns=[0])

In [14]:
df.href

0         https://edadeal.ru/lipeck/retailers/perekrestok
1         https://edadeal.ru/lipeck/retailers/perekrestok
2         https://edadeal.ru/lipeck/retailers/perekrestok
3         https://edadeal.ru/lipeck/retailers/perekrestok
4         https://edadeal.ru/lipeck/retailers/perekrestok
                              ...                        
3648    https://edadeal.ru/lipeck/retailers/gulliver-m...
3649    https://edadeal.ru/lipeck/retailers/gulliver-m...
3650    https://edadeal.ru/lipeck/retailers/gulliver-m...
3651    https://edadeal.ru/lipeck/retailers/gulliver-m...
3652    https://edadeal.ru/lipeck/retailers/gulliver-m...
Name: href, Length: 3653, dtype: object

In [11]:
df = df.drop(df[df['data'].isnull()].index).reset_index()
tmp = df['data'].apply(lambda x: x.get('items', np.nan))
df['data'] = pd.Series(np.where(tmp, tmp, np.nan))

In [13]:
df = df.explode('data', ignore_index=True)
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1077621 entries, 0 to 1077620
Data columns (total 12 columns):
 #   Column     Non-Null Count    Dtype 
---  ------     --------------    ----- 
 0   index      1077621 non-null  int64 
 1   sity       1077621 non-null  object
 2   title      1077621 non-null  object
 3   discounts  1077621 non-null  int64 
 4   alcohol    1077621 non-null  bool  
 5   href       1077621 non-null  object
 6   su1        1077621 non-null  object
 7   su2        1077621 non-null  object
 8   su3        818839 non-null   object
 9   page_href  1077621 non-null  object
 10  su_id      1077621 non-null  object
 11  data       1073032 non-null  object
dtypes: bool(1), int64(2), object(9)
memory usage: 91.5+ MB


In [127]:
def process_column(x):
    if isinstance(x, dict):
        title = x['title']
        sku_id = x.get('uuid', np.nan)
        dateStart = pd.to_datetime(x.get('dateStart', np.nan), unit='ms')
        dateEnd = pd.to_datetime(x.get('dateEnd', np.nan), unit='ms')
        price = x['priceData'].get('new', {}).get('from', np.nan)
        price_from = x['priceData'].get('new', {}).get('to', np.nan)
        price_to = x['priceData'].get('new', {}).get('value', np.nan)
        discountPercent = x['discountPercent']
        quantity = x.get('quantity', np.nan)
        quantityUnit = x.get('quantityUnit', np.nan)
        
        return pd.Series([title, sku_id, dateStart, dateEnd, price, price_from, price_to, discountPercent, quantity, quantityUnit])
    return pd.Series([np.nan, np.nan, np.nan, np.nan, np.nan, np.nan, np.nan])

now_columns = df['data'].progress_apply(process_column)
df[['title_unit', 'sku_id', 'dateStart', 'dateEnd', 'price', 'price_from', 'price_to', 'discountPercent', 'quantity', 'quantityUnit']] = now_columns
df.info()

100%|██████████████████████████████████████████████████████████████████████████| 91775/91775 [00:35<00:00, 2560.67it/s]

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 91775 entries, 0 to 91774
Data columns (total 23 columns):
 #   Column           Non-Null Count  Dtype         
---  ------           --------------  -----         
 0   level_0          91775 non-null  int64         
 1   index            91775 non-null  int64         
 2   sity             91775 non-null  object        
 3   title            91775 non-null  object        
 4   discounts        91775 non-null  int64         
 5   alcohol          91775 non-null  bool          
 6   href             91775 non-null  object        
 7   su1              91775 non-null  object        
 8   su2              91775 non-null  object        
 9   su3              60425 non-null  object        
 10  page_href        91775 non-null  object        
 11  su_id            91775 non-null  object        
 12  data             90641 non-null  object        
 13  title_unit       90641 non-null  object        
 14  sku_id           90641 non-null  objec

In [41]:
test = (df[df['title'] == 'Пятёрочка']['data']).copy().reset_index(drop=True).to_frame()
test['name'] = test['data'].apply(lambda x: x.get('title', np.nan))
test

,data,name
0,"{'type': 'meta_offer', 'itemType': 'meta_offer...",Лопатка свиная Курский МП в маринаде 1.3 кг
1,"{'type': 'meta_offer', 'itemType': 'meta_offer...",Купаты Магнит Свежесть cвиные охлажденные 400г
2,"{'type': 'meta_offer', 'itemType': 'meta_offer...",Колбаски Мираторг Немецкие 400г
3,"{'type': 'meta_offer', 'itemType': 'meta_offer...",Купаты Агроэко Экстра с томатами 400г
4,"{'type': 'meta_offer', 'itemType': 'meta_offer...",Фарш Агро-Белогорье из свинины полуфабрикат 700г
...,...,...
17259,"{'type': 'meta_offer', 'itemType': 'meta_offer...",Влажный корм для кошек Perfect Fit Иммунитет с...
17260,"{'type': 'meta_offer', 'itemType': 'meta_offer...",Влажный корм для кошек Sheba Craft Collection ...
17261,"{'type': 'offer', 'itemType': 'offer', 'view':...",Влажный корм для кошек Whiskas полнорационный ...
17262,"{'type': 'offer', 'itemType': 'offer', 'view':...",Влажный корм для кошек Perfect Fit для стерили...


In [48]:
test['name'].str.contains(r'Влажный корм для').sum()

666